In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import polars as pl

grid_questions = pl.read_parquet(
    "hf://datasets/allenai/ZebraLogicBench-private/grid_mode/test-00000-of-00001.parquet"
).to_dicts()
grid_questions

[{'id': 'lgp-test-5x6-16',
  'size': '5*6',
  'puzzle': 'There are 5 houses, numbered 1 to 5 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:\n - Each person has a unique name: `Peter`, `Alice`, `Bob`, `Eric`, `Arnold`\n - The people are of nationalities: `norwegian`, `german`, `dane`, `brit`, `swede`\n - People have unique favorite book genres: `fantasy`, `biography`, `romance`, `mystery`, `science fiction`\n - Everyone has something unique for lunch: `stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`\n - Each person has a favorite color: `red`, `green`, `blue`, `yellow`, `white`\n - The people keep unique animals: `bird`, `dog`, `cat`, `horse`, `fish`\n\n## Clues:\n1. The person who loves fantasy books is the Norwegian.\n2. The cat lover and the person who loves biography books are next to each other.\n3. The German is Bob.\n4. The person who loves yel

In [28]:
from lib.rl.episode import Episode, EpisodeCompletion
import re


question = grid_questions[0]

prompt = f"""
{question["puzzle"]}
Fill in the grid with the correct values:

| {' | '.join(question["solution"]["header"])} |
| {' | '.join(["-" * len(header) for header in question["solution"]["header"]])} |
"""

for _ in question["solution"]["rows"]:
    prompt += f"| {' | '.join([" " * len(header) for header in question["solution"]["header"]])} |\n"

print(prompt)

pattern = re.compile(
    r"\| " + r"\|".join(r"(.*?)" for _ in question["solution"]["header"]) + r" \|"
)


def on_sample(completions: list[EpisodeCompletion]):
    for completion in completions:
        assert "content" in completion.last_assistant_message and isinstance(
            completion.last_assistant_message["content"], str
        )
        num_cells = sum(len(row) for row in question["solution"]["rows"])
        num_correct = 0
        for match, row in zip(
            re.findall(pattern, completion.last_assistant_message["content"])[
                -len(question["solution"]["rows"]) :
            ],
            question["solution"]["rows"],
        ):
            for cell, value in zip(match, row):
                if cell.strip().lower() == value.lower():
                    num_correct += 1
        completion.commit(reward=num_correct / num_cells)


episode = Episode(
    messages=[{"role": "user", "content": prompt}],
    on_sample=on_sample,
)


There are 5 houses, numbered 1 to 5 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Alice`, `Bob`, `Eric`, `Arnold`
 - The people are of nationalities: `norwegian`, `german`, `dane`, `brit`, `swede`
 - People have unique favorite book genres: `fantasy`, `biography`, `romance`, `mystery`, `science fiction`
 - Everyone has something unique for lunch: `stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`
 - Each person has a favorite color: `red`, `green`, `blue`, `yellow`, `white`
 - The people keep unique animals: `bird`, `dog`, `cat`, `horse`, `fish`

## Clues:
1. The person who loves fantasy books is the Norwegian.
2. The cat lover and the person who loves biography books are next to each other.
3. The German is Bob.
4. The person who loves yellow is Bob.
5. The person whose favorite color is green is Peter.
6.

In [33]:
question["solution"]

{'header': ['House',
  'Name',
  'Nationality',
  'BookGenre',
  'Food',
  'Color',
  'Animal'],
 'rows': [['1', 'Bob', 'german', 'mystery', 'grilled cheese', 'yellow', 'dog'],
  ['2', 'Eric', 'norwegian', 'fantasy', 'stew', 'blue', 'fish'],
  ['3', 'Peter', 'dane', 'science fiction', 'spaghetti', 'green', 'cat'],
  ['4', 'Arnold', 'swede', 'biography', 'stir fry', 'red', 'bird'],
  ['5', 'Alice', 'brit', 'romance', 'pizza', 'white', 'horse']]}

In [32]:
class MockEpisodeCompletion:
    def __init__(self, last_assistant_message):
        self.last_assistant_message = last_assistant_message

    def commit(self, reward):
        print(f"committing reward {reward}")


episode.on_sample(
    [
        MockEpisodeCompletion(
            {
                "content": """
| House | Name   | Nationality | Book Genre      | Food           | Color  | Animal |
|-------|--------|-------------|-----------------|----------------|--------|--------|
| 1     | Eric   | Brit        | Romance         | Grilled Cheese | Red    | Bird   |
| 2     | Peter  | Norwegian   | Fantasy         | Stew           | Green  | Cat    |
| 3     | Bob    | German      | Biography       | Pizza          | Yellow | Dog    |
| 4     | Arnold | Swede       | Mystery         | Stir Fry       | Blue   | Fish   |
| 5     | Alice  | Dane        | Science Fiction | Spaghetti      | White  | Horses |
"""
            }
        )
    ]
)

committing reward 0.4


In [17]:
"-" * len(question["solution"]["header"])

'-------'

In [1]:
from constraint import Problem, AllDifferentConstraint

# Define the problem
problem = Problem()

# Define locations and timeslots
locations = ["Town Square", "Beachfront", "Community Hall"]
timeslots = ["10:00-12:00", "12:30-2:30", "3:00-5:00"]

# Define events and their constraints
events = ["Parade", "Sandcastle Contest", "Talent Show", 
          "Fireworks", "Artisan Market", "Charity Auction"]

performers = ["Alice", "Ben", "Carlos", "Daisy"]

# Assign each event a (location, timeslot) pair
for event in events:
    problem.addVariable(event, [(loc, time) for loc in locations for time in timeslots])

# Add location-specific constraints
problem.addConstraint(lambda x: x[0] == "Town Square", ["Parade"])
problem.addConstraint(lambda x: x[0] == "Beachfront", ["Sandcastle Contest"])
problem.addConstraint(lambda t1, t2: t1 != t2, ["Talent Show", "Charity Auction"])

# Performer availability constraints
problem.addVariable("Carlos_Event", events)
problem.addConstraint(lambda e1, e2: e1 != e2, ["Carlos_Event", "Fireworks"])  # Carlos performs only once

def available_before_3(event):
    return event[1] in ["10:00-12:00", "12:30-2:30"]

problem.addConstraint(available_before_3, ["Alice", "Ben"])

# Daisy participation constraints
problem.addConstraint(lambda p, m: p == ("Town Square", "10:00-12:00") and m[0] == "Community Hall", ["Parade", "Artisan Market"])

# Time gap constraints
def no_overlap(e1, e2):
    return e1[1] != e2[1]

for location in locations:
    events_in_location = [event for event in events if location in event]
    problem.addConstraint(AllDifferentConstraint(), events_in_location)

# Sandcastle Contest must overlap with another event for 1 hour
problem.addConstraint(lambda s, o: s[1] == o[1], ["Sandcastle Contest", "Parade"])

# Fireworks must be the final event
problem.addConstraint(lambda f: f[1] == "3:00-5:00", ["Fireworks"])

# Solve the problem
solution = problem.getSolutions()

# Display the solution(s)
if solution:
    for sol in solution:
        print("Valid Schedule:")
        for event in events:
            print(f"{event}: Location - {sol[event][0]}, Time - {sol[event][1]}")
else:
    print("No valid solution found.")


KeyError: 'Alice'

In [7]:
from constraint import Problem, AllDifferentConstraint

# Create a new problem instance
problem = Problem()

# Define the possible colors
colors = ["Red", "Blue", "Green"]

# Add variables for each friend with the possible colors
problem.addVariable("Alice", colors)
problem.addVariable("Bob", colors)
problem.addVariable("Charlie", colors)

# Add a constraint that all shirts are different colors
problem.addConstraint(AllDifferentConstraint())

# Constraint 1: Alice is not wearing the Red shirt
problem.addConstraint(lambda alice: alice != "Red", ["Alice"])

# Constraint 2: Bob is wearing either the Blue or Green shirt
problem.addConstraint(lambda bob: bob in ["Blue", "Green"], ["Bob"])

# Constraint 3: Charlie is wearing the Red shirt if and only if Bob is not wearing the Green shirt
def charlie_constraint(charlie, bob):
    return (charlie == "Red") == (bob != "Green")

problem.addConstraint(charlie_constraint, ["Charlie", "Bob"])

# Find and print all possible solutions
solutions = problem.getSolutions()
for solution in solutions:
    print(solution)

{'Bob': 'Blue', 'Charlie': 'Red', 'Alice': 'Green'}


In [8]:
# Create a new problem instance
problem = Problem()

# Define the possible favorite fruits
fruits = ["Apple", "Banana", "Cherry"]

# Add variables for each colleague with the possible fruits
problem.addVariable("Diana", fruits)
problem.addVariable("Edward", fruits)
problem.addVariable("Fiona", fruits)

# Add a constraint that all favorite fruits are different
problem.addConstraint(AllDifferentConstraint())

# Constraint 1: Diana does not like Banana
problem.addConstraint(lambda diana: diana != "Banana", ["Diana"])

# Constraint 2: Edward's favorite fruit is either Apple or Cherry
problem.addConstraint(lambda edward: edward in ["Apple", "Cherry"], ["Edward"])

# Constraint 3: Fiona likes Cherry if and only if Edward does not like Apple
def fiona_constraint(fiona, edward):
    return (fiona == "Cherry") == (edward != "Apple")

problem.addConstraint(fiona_constraint, ["Fiona", "Edward"])

# Find and print all possible solutions
solutions = problem.getSolutions()
for solution in solutions:
    print(solution)

{'Edward': 'Apple', 'Fiona': 'Banana', 'Diana': 'Cherry'}


In [11]:
from constraint import Problem, AllDifferentConstraint

# Create a new problem instance
problem = Problem()

# Define the possible roles
roles = ["Developer", "Designer", "Tester", "Project Manager"]

# Add variables for each colleague with the possible roles
problem.addVariable("Grace", roles)
problem.addVariable("Henry", roles)
problem.addVariable("Irene", roles)
problem.addVariable("Jack", roles)

# Add a constraint that all roles are assigned uniquely
problem.addConstraint(AllDifferentConstraint())

# Constraint 1: Grace cannot be the Project Manager
problem.addConstraint(lambda grace: grace != "Project Manager", ["Grace"])

# Constraint 2: Henry prefers not to be the Tester
problem.addConstraint(lambda henry: henry != "Tester", ["Henry"])

# Constraint 3: Grace and Henry cannot be Developer and Designer in any order
def dev_designer_not_grace_henry(grace, henry):
    return not (
        (grace == "Developer" and henry == "Designer") or
        (grace == "Designer" and henry == "Developer")
    )

problem.addConstraint(dev_designer_not_grace_henry, ["Grace", "Henry"])

# Constraint 4: Irene will only take the role of Tester if Jack is the Project Manager
def irene_tester_jack_pm(irene, jack):
    if irene == "Tester":
        return jack == "Project Manager"
    return True

problem.addConstraint(irene_tester_jack_pm, ["Irene", "Jack"])

# Constraint 5: Jack cannot be the Designer
problem.addConstraint(lambda jack: jack != "Designer", ["Jack"])

# Constraint 6 (Revised): The Project Manager must be either Grace or Irene
def project_manager_is_grace_or_irene(grace, irene):
    return grace == "Project Manager" or irene == "Project Manager"

problem.addConstraint(project_manager_is_grace_or_irene, ["Grace", "Irene"])

# Constraint 7: Henry is either the Developer or the Project Manager
problem.addConstraint(lambda henry: henry in ["Developer", "Project Manager"], ["Henry"])

# Constraint 8: Grace will take the role of Designer only if Irene is not the Tester
def grace_designer_if_irene_not_tester(grace, irene):
    if grace == "Designer":
        return irene != "Tester"
    return True

problem.addConstraint(grace_designer_if_irene_not_tester, ["Grace", "Irene"])

# The problem is now fully defined. You can run the solver to find solutions.
# Here's how you can retrieve and print the solutions:

solutions = problem.getSolutions()
for idx, solution in enumerate(solutions, start=1):
    print(f"Solution {idx}: {solution}")

In [13]:
from constraint import Problem, AllDifferentConstraint

# Create a new problem instance
problem = Problem()

# Define the possible roles
roles = ["Developer", "Designer", "Tester", "Project Manager"]

# Add variables for each colleague with the possible roles
problem.addVariable("Grace", roles)
problem.addVariable("Henry", roles)
problem.addVariable("Irene", roles)
problem.addVariable("Jack", roles)

# Add a constraint that all roles are assigned uniquely
problem.addConstraint(AllDifferentConstraint())

# Constraint 1: Grace cannot be the Project Manager
problem.addConstraint(lambda grace: grace != "Project Manager", ["Grace"])

# Constraint 2: Henry prefers not to be the Tester
problem.addConstraint(lambda henry: henry != "Tester", ["Henry"])

# Constraint 5: Jack cannot be the Designer
problem.addConstraint(lambda jack: jack != "Designer", ["Jack"])

# Constraint 6: Irene must be the Project Manager
problem.addConstraint(lambda irene: irene == "Project Manager", ["Irene"])

# Constraint 7: Henry is either the Developer or the Project Manager
# Since Irene is Project Manager, Henry must be Developer
def henry_role(henry, irene):
    if irene == "Project Manager":
        return henry == "Developer"
    return henry in ["Developer", "Project Manager"]

problem.addConstraint(henry_role, ["Henry", "Irene"])

# Constraint 8: Grace will take the role of Designer only if Irene is not the Tester
# Since Irene is Project Manager, she is not Tester, so Grace can be Designer
# However, we include the constraint for completeness
def grace_designer_if_irene_not_tester(grace, irene):
    if grace == "Designer":
        return irene != "Tester"
    return True

problem.addConstraint(grace_designer_if_irene_not_tester, ["Grace", "Irene"])

# Now, the problem is fully defined. You can run the solver to find solutions.
# Here's how you can retrieve and print the solutions:

solutions = problem.getSolutions()
for idx, solution in enumerate(solutions, start=1):
    print(f"Solution {idx}: {solution}")

Solution 1: {'Irene': 'Project Manager', 'Henry': 'Developer', 'Grace': 'Designer', 'Jack': 'Tester'}


In [22]:
import constraint

problem = constraint.Problem()
problem.addVariable("A", [1, 2, 3])
problem.addVariable("B", [4, 5, 6])
problem.addVariable("C", [7, 8, 9])
problem.addConstraint(lambda a, b: a + b == 7, ("A", "B"))
problem.addConstraint(lambda b, c: b - c == -3, ("B", "C"))
problem.addConstraint(lambda a, c: a * c == 16, ("A", "C"))
solutions = problem.getSolutions()
print(solutions)

[{'A': 2, 'B': 5, 'C': 8}]


In [14]:
import inspect

inspect.getsource(problem._constraints[0][0]._func)

'problem.addConstraint(lambda a, b: a + b == 7, ("A", "B"))\n'

In [ ]:
import random

def generate_problem() -> None:
    problem = constraint.Problem()
    for _ in range(10):
        problem.addVariable(random.choice("ABCDEFGHIJ"), [1, 2, 3])